In [64]:
# Third Party libraries
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

from sklearn.linear_model import LinearRegression

In [65]:
# Import Data_cleaning module
from modules.data_clean import DataCleaner

# Load DF
df = pd.read_csv('../../data/raw/Fire_Incidents_Data.csv', low_memory=False)

In [66]:
# Cleanse Dataframe
df = DataCleaner.cleanse_dataframe(df)

In [67]:
df

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
0,81 - Engine Area,<NA>,<NA>,0.0,0.0,15000.0,NaN,<NA>,01 - Fire,<NA>,<NA>,<NA>,999 - Undetermined,1.0,Vehicle Fire,47 - Vehicle,1 - Extinguished by fire department,1.0,4.0,99 - Undetermined,"896 - Sidewalk, street, roadway, highway, hwy ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"7 - Fully involved (total structure, vehicle, ...",0.0
1,"75 - Trash, rubbish area (outside)",<NA>,<NA>,0.0,0.0,50.0,NaN,<NA>,01 - Fire,<NA>,<NA>,<NA>,999 - Undetermined,18.0,Fire - Grass/Rubbish,97 - Other,1 - Extinguished by fire department,1.0,4.0,03 - Suspected Vandalism,"896 - Sidewalk, street, roadway, highway, hwy ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2 - Fire with no evidence from street,0.0
3,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,1 - Confined to object of origin,01 - Fire,9 - Undetermined,8 - Not applicable (no system),9 - Undetermined,999 - Undetermined,5.0,Fire - Commercial/Industrial,99 - Undetermined (formerly 98),1 - Extinguished by fire department,6.0,22.0,99 - Undetermined,511 - Department Store,9 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,9 - Type undetermined,"8 - Not applicable: No alarm, no persons present",99 - Undetermined,8 - Not applicable - no sprinkler system present,9 - Undetermined,3 - Fire with smoke showing only - including v...,0.0
5,81 - Engine Area,<NA>,<NA>,0.0,0.0,1500.0,NaN,<NA>,01 - Fire,<NA>,<NA>,<NA>,999 - Undetermined,25.0,Vehicle Fire,47 - Vehicle,1 - Extinguished by fire department,7.0,25.0,99 - Undetermined,837 - Vehicles or Vehicle Parts,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4 - Flames showing from small area (one storey...,0.0
6,22 - Sleeping Area or Bedroom (inc. patients r...,01 - Normal (no change),1 - No business interruption,0.0,0.0,2000.0,0.0,2 - Confined to part of room/area of origin,01 - Fire,"8 - Not applicable: No fire alarm system, no p...",8 - Not applicable (no system),8 - Not applicable (bldg not classified by OBC...,"51 - Incandescent Lamp - Light Bulb, Spotlight",19.0,Fire - Residential,16 - Insulation,1 - Extinguished by fire department,6.0,22.0,20 - Design/Construction/Installation/Maintena...,301 - Detached Dwelling,1 - Floor/suite of fire origin: No smoke alarm,98 - Not applicable: Alarm operated OR presenc...,8 - Not applicable - no smoke alarm or presenc...,7 - Not applicable: Occupant(s) first alerted ...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,2 - Fire with no evidence from street,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29419,81 - Engine Area,<NA>,<NA>,NaN,0.0,50000.0,NaN,<NA>,01 - Fire,<NA>,<NA>,<NA>,82 - Vehicle - Mechanical,2.0,VEFH,Miscellaneous,1 - Extinguished by fire department,5.0,18.0,Mechanical/Electrical Failure,"896 - Sidewalk, street, roadway, highway, hwy ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3 - Fire with smoke showing only - including v...,0.0
29420,42 - Garage,01 - Normal (no change),1 - No business interruption,NaN,0.0,0.0,0.0,1 - Confined to object of origin,01 - Fire,3 - No one (at risk) evacuated as a result of ...,1 - Fire alarm system o

### Defining all features
- Numerical Columns
- Categorial Columns
    - Ordinal Columns
    - Onehot Columns

In [68]:
NUMERICAL_COLS = [
    "Civilian_Casualties",
    "Count_of_Persons_Rescued",
    "Estimated_Dollar_Loss",
    "Estimated_Number_Of_Persons_Displaced",
    "Number_of_responding_apparatus",
    "Number_of_responding_personnel",
    "TFS_Firefighter_Casualties"
]

CATEGORICAL_COLS = ['Area_of_Origin', 'Building_Status', 'Business_Impact', 'Extent_Of_Fire', 'Final_Incident_Type', 
                'Fire_Alarm_System_Impact_on_Evacuation', 'Fire_Alarm_System_Operation', 'Fire_Alarm_System_Presence', 
                'Ignition_Source', 'Initial_CAD_Event_Type', 'Material_First_Ignited', 'Method_Of_Fire_Control', 
                'Possible_Cause', 'Property_Use', 'Smoke_Alarm_at_Fire_Origin', 'Smoke_Alarm_at_Fire_Origin_Alarm_Failure', 
                'Smoke_Alarm_at_Fire_Origin_Alarm_Type', 'Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation', 
                'Smoke_Spread', 'Sprinkler_System_Operation', 'Sprinkler_System_Presence', 'Status_of_Fire_On_Arrival', 'Incident_Ward']

# ['Business_Impact', 'Extent_Of_Fire', 'Sprinkler_System_Presence', 'Status_of_Fire_On_Arrival']
ORDINAL_COLS = CATEGORICAL_COLS[2:4] + CATEGORICAL_COLS[20:22]

# Everything else.
ONEHOT_COLS = list(set(CATEGORICAL_COLS) - set(ORDINAL_COLS))

### Imputation

#### Categorical Imputation

Will use SimpleImputer, as it can work with strings.

In [69]:
# Specify pd.NA as the missing value indicator for SimpleImputer
imputer = SimpleImputer(strategy='most_frequent', missing_values=pd.NA)

# Apply the imputer to the categorical columns
df[CATEGORICAL_COLS] = imputer.fit_transform(df[CATEGORICAL_COLS])

#### Numerical imputation.

This uses KNN for numerical columns missing data.

In [70]:
import numpy as np

# Replace pd.NA with np.nan
# KNNImputer does not directly support pd.NA
df = df.replace(pd.NA, np.nan)

imputer = KNNImputer()

# Apply the imputer to the categorical columns
df[NUMERICAL_COLS] = imputer.fit_transform(df[NUMERICAL_COLS])

df

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
0,81 - Engine Area,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,15000.0,0.2,1 - Confined to object of origin,01 - Fire,"8 - Not applicable: No fire alarm system, no p...",1 - Fire alarm system operated,1 - Fire alarm system present,999 - Undetermined,1.0,Vehicle Fire,47 - Vehicle,1 - Extinguished by fire department,1.0,4.0,99 - Undetermined,"896 - Sidewalk, street, roadway, highway, hwy ...",2 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,8 - Not applicable - no smoke alarm or presenc...,1 - All persons (at risk of injury) self evacu...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,"7 - Fully involved (total structure, vehicle, ...",0.0
1,"75 - Trash, rubbish area (outside)",01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,50.0,0.0,1 - Confined to object of origin,01 - Fire,"8 - Not applicable: No fire alarm system, no p...",1 - Fire alarm system operated,1 - Fire alarm system present,999 - Undetermined,18.0,Fire - Grass/Rubbish,97 - Other,1 - Extinguished by fire department,1.0,4.0,03 - Suspected Vandalism,"896 - Sidewalk, street, roadway, highway, hwy ...",2 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,8 - Not applicable - no smoke alarm or presenc...,1 - All persons (at risk of injury) self evacu...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,2 - Fire with no evidence from street,0.0
3,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,1 - Confined to object of origin,01 - Fire,9 - Undetermined,8 - Not applicable (no system),9 - Undetermined,999 - Undetermined,5.0,Fire - Commercial/Industrial,99 - Undetermined (formerly 98),1 - Extinguished by fire department,6.0,22.0,99 - Undetermined,511 - Department Store,9 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,9 - Type undetermined,"8 - Not applicable: No alarm, no persons present",99 - Undetermined,8 - Not applicable - no sprinkler system present,9 - Undetermined,3 - Fire with smoke showing only - including v...,0.0
5,81 - Engine Area,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,1500.0,0.0,1 - Confined to object of origin,01 - Fire,"8 - Not applicable: No fire alarm system, no p...",1 - Fire alarm system operated,1 - Fire alarm system present,999 - Undetermined,25.0,Vehicle Fire,47 - Vehicle,1 - Extinguished by fire department,7.0,25.0,99 - Undetermined,837 - Vehicles or Vehicle Parts,2 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,8 - Not applicable - no smoke alarm or presenc...,1 - All persons (at risk of injury) self evacu...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,4 - Flames showing from small area (one storey...,0.0
6,22 - Sleeping Area or Bedroom (inc. patients r...,01 - Normal (no change),1 - No business interruption,0.0,0.0,2000.0,0.0,2 - Confined to part of room/area of origin,01 - Fir

### Categorical Encoding

Encoding all categorical columns as ordinal.

This yields bad results.

In [71]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np

# Specify ordinal encoder
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)

# Apply the encoder to specified columns
df[CATEGORICAL_COLS] = ordinal_encoder.fit_transform(df[CATEGORICAL_COLS])
df

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
0,60.0,0.0,5.0,0.0,0.0,15000.0,0.2,0.0,0.0,5.0,0.0,0.0,83.0,1.0,126.0,25.0,0.0,1.0,4.0,22.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,5.0,0.0
1,56.0,0.0,5.0,0.0,0.0,50.0,0.0,0.0,0.0,5.0,0.0,0.0,83.0,18.0,62.0,51.0,0.0,1.0,4.0,2.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,1.0,0.0
3,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,3.0,83.0,5.0,60.0,52.0,0.0,6.0,22.0,22.0,138.0,8.0,9.0,5.0,5.0,9.0,5.0,3.0,2.0,0.0
5,60.0,0.0,5.0,0.0,0.0,1500.0,0.0,0.0,0.0,5.0,0.0,0.0,83.0,25.0,126.0,25.0,0.0,7.0,25.0,22.0,281.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,3.0,0.0
6,6.0,0.0,0.0,0.0,0.0,2000.0,0.0,3.0,0.0,5.0,2.0,2.0,47.0,19.0,72.0,5.0,0.0,6.0,22.0,6.0,71.0,0.0,9.0,4.0,4.0,1.0,5.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29419,60.0,0.0,5.0,0.0,0.0,50000.0,1.6,0.0,0.0,5.0,0.0,0.0,69.0,2.0,115.0,59.0,0.0,5.0,18.0,26.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,2.0,0.0
29420,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,83.0,13.0,26.0,59.0,0.0,5.0,17.0,28.0,81.0,3.0,9.0,3.0,2.0,2.0,2.0,0.0,1.0,0.0
29421,60.0,0.0,5.0,0.0,0.0,5000.0,0.0,0.0,0.0,5.0,0.0,0.0,68.0,20.0,114.0,59.0,0.0,1.0,4.0,26.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,2.0,0.0
29422,25.0,0.0,5.0,0.2,0.0,2000.0,0.0,3.0,0.0,2.0,0.0,0.0,83.0,13.0,26.0,61.0,0.0,8.0,26.0,27.0,81.0,3.0,9.0,3.0,5.0,3.0,1.0,0.0,1.0,0.0


Encoding onehot columns - Work in progress/Not working.

In [72]:


# onehot_encoder = OneHotEncoder(handle_unknown="ignore")
# # Apply the imputer to the categorical columns
# df_encoded= onehot_encoder.fit_transform(df[ONEHOT_COLS])

# df_encoded

# from sklearn.preprocessing import OneHotEncoder
# from sklearn.linear_model import LinearRegression

# # X is a numpy array with your features
# # y is the label array
# enc = OneHotEncoder(sparse=False)
# X_transform = enc.fit_transform(X)

# # apply your linear regression as you want
# model = LinearRegression()
# model.fit(X_transform, y)

# print("Mean squared error: %.2f" % np.mean((model.predict(X_transform) - y) ** 2))

### Standard scaler on numerical Data

In [73]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the DataFrame
df[NUMERICAL_COLS] = scaler.fit_transform(df[NUMERICAL_COLS])
df

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
0,60.0,0.0,5.0,-0.222403,-0.052852,-0.054726,-0.124133,0.0,0.0,5.0,0.0,0.0,83.0,1.0,126.0,25.0,0.0,-0.835006,-0.865484,22.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,5.0,-0.099823
1,56.0,0.0,5.0,-0.222403,-0.052852,-0.091090,-0.126145,0.0,0.0,5.0,0.0,0.0,83.0,18.0,62.0,51.0,0.0,-0.835006,-0.865484,2.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,1.0,-0.099823
3,56.0,0.0,0.0,-0.222403,-0.052852,-0.091211,-0.126145,0.0,0.0,6.0,2.0,3.0,83.0,5.0,60.0,52.0,0.0,-0.212291,-0.144944,22.0,138.0,8.0,9.0,5.0,5.0,9.0,5.0,3.0,2.0,-0.099823
5,60.0,0.0,5.0,-0.222403,-0.052852,-0.087563,-0.126145,0.0,0.0,5.0,0.0,0.0,83.0,25.0,126.0,25.0,0.0,-0.087748,-0.024854,22.0,281.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,3.0,-0.099823
6,6.0,0.0,0.0,-0.222403,-0.052852,-0.086347,-0.126145,3.0,0.0,5.0,2.0,2.0,47.0,19.0,72.0,5.0,0.0,-0.212291,-0.144944,6.0,71.0,0.0,9.0,4.0,4.0,1.0,5.0,2.0,1.0,-0.099823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29419,60.0,0.0,5.0,-0.222403,-0.052852,0.030407,-0.110048,0.0,0.0,5.0,0.0,0.0,69.0,2.0,115.0,59.0,0.0,-0.336834,-0.305064,26.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,2.0,-0.099823
29420,23.0,0.0,0.0,-0.222403,-0.052852,-0.091211,-0.126145,0.0,0.0,2.0,0.0,0.0,83.0,13.0,26.0,59.0,0.0,-0.336834,-0.345094,28.0,81.0,3.0,9.0,3.0,2.0,2.0,2.0,0.0,1.0,-0.099823
29421,60.0,0.0,5.0,-0.222403,-0.052852,-0.079050,-0.126145,0.0,0.0,5.0,0.0,0.0,68.0,20.0,114.0,59.0,0.0,-0.835006,-0.865484,26.0,313.0,2.0,9.0,4.0,0.0,1.0,5.0,2.0,2.0,-0.099823
29422,25.0,0.0,5.0,0.314813,-0.052852,-0.086347,-0.126145,3.0,0.0,2.0,0.0,0.0,83.0,13.0,26.0,61.0,0.0,0.036794,0.015176,27.0,81.0,3.0,9.0,3.0,5.0,3.0,1.0,0.0,1.0,-0.099823


### Train test split

In [74]:
from sklearn.model_selection import train_test_split

# List all columns in the DataFrame
all_columns = df.columns.tolist()

# Use every other column in the df except for the response variable
features = [col for col in all_columns if col != 'Estimated_Dollar_Loss']

# Separate features from response variable
X, y = df[features], df['Estimated_Dollar_Loss']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=12)

### Apply logistic regressor (no pipeline)

#### Helper Function

In [75]:
# Function: Print Results of a Model (R2, MEAN ABSOLUTE ERROR, MEAN SQUARED ERROR, COEF and INTERCEPT)

def print_results(r2, mae, mse, coefficients, intercept):
    print(f"R-squared score: {r2:.4f}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Squared Error: {mse}")
    print(f"Coefficients: {coefficients}")
    print(f"Intercept: {intercept}")

# Function: Return Results (R2, MEAN ABSOLUTE ERROR, MEAN SQUARED ERROR, COEF and INTERCEPT)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def get_results(model, y_test, y_pred):
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    coefficients = model.coef_
    intercept = model.intercept_
    return (r2, mae, mse, coefficients, intercept)

In [76]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

lr = LinearRegression() # to do - settings hyperparameters 
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)
residuals_lr = y_test - y_pred_lr

(r2_lr, mae_lr, mse_lr, coefficients_lr, intercept_lr) = get_results(lr, y_test, y_pred_lr)

# Results - Another option would be to use statsmodels to display a summary
print("---------MLR----------")
print_results(r2_lr, mae_lr, mse_lr, coefficients_lr, intercept_lr)
print("----------------------")

---------MLR----------
R-squared score: -0.1977
Mean Absolute Error: 0.2562080171939384
Mean Squared Error: 0.4438193139362556
Coefficients: [ 2.52945446e-03  1.13152685e-02 -1.84693101e-03 -4.46665309e-02
 -3.84788170e-03  3.15442991e-02 -2.08194071e-02 -6.74607950e-02
  2.82034752e-02 -2.00447760e-03 -1.89178430e-02  1.33138699e-04
  2.27749759e-04  1.73448395e-03  6.69429036e-04  5.03808900e-02
  7.74134933e-01 -1.72223390e-01 -3.01146203e-03  4.47986996e-04
  4.25595358e-03  2.98537259e-03  1.37502567e-02 -1.05967570e-02
 -7.17293189e-03  3.31091936e-03  8.44133994e-03 -1.51292218e-02
 -3.08112685e-02]
Intercept: -0.36687439705087116
----------------------


### Create mini-pipeline
Applies the following:
- Ordinal encoding
- One hot encoding
- Imputes the data 
- Logistic regressor

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Create a pipeline with ColumnTransformer, OneHotEncoder, and LogisticRegression
# pipeline = Pipeline([
#     ('preprocessor', ColumnTransformer([
#         ('ordinal', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan), ORDINAL_COLS),
#         ('onehot', OneHotEncoder(handle_unknown='ignore'), ONEHOT_COLS),
#         ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA), ONEHOT_COLS + ORDINAL_COLS)
#     ], remainder='passthrough')),
#     ('classifier', LogisticRegression(max_iter=1000))
# ])


### Apply Logistic Regressor (pipeline)

In [ ]:

pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the model (e.g., using accuracy score)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)